In [1]:
import PIL as pil
from PIL import Image
import math

#### Part I

Main modification from prior versions: User-determined # of columns and rows will almost always not divde perfectly into the pixel dimensions.  Rounding up or down to utilize a single window size (or fixed # of pixels for all column/rows) causes total number of cols/rows produced to vary from the input value. Instead - since cropping is undesirable - round each individual col/row size to the nearest integer.  
Result: each column may be +/- 2 pixel in width from its neighbor, but there will not be any extra columns added, nor one considerably different-sized column at the end.

In [2]:
def img1_tileRGBs(path, num_cols, num_rows):

    pic1 = pil.Image.open(path).convert('RGB')
    width1, height1 = pic1.size
    
    def variable_steps(max_pixels, row_or_col):
        px_per_dim = max_pixels/row_or_col #not rounded
        tups = []
        start = 0
        end = start + px_per_dim #not rounded
    
        for i in range(0,max_pixels+1):
            tups.append( ( int(round(start,0)), int(round(end,0)) ) ) #round nearest int here
            start = end
            end = start + px_per_dim

            if tups[-1][1] >= max_pixels:
                break
        
        return tups
    
    
    chunk_it_up_cols = variable_steps(width1, num_cols)
    chunk_it_up_rows = variable_steps(height1, num_rows)
    
    #find center pixel in each col and row, round nearest int
    col_centers = [int(round((i[0]+i[1])/2,0)) for i in chunk_it_up_cols]
    row_centers = [int(round((i[0]+i[1])/2,0)) for i in chunk_it_up_rows]
    #knit into tuple, coordinate of px at the center of each tile (or closest to true center)
    center_px_coords = [(x,y) for x in col_centers for y in row_centers]
    #the order of center_px_coords is (col_0,row_0), (col_0,row_1), ... on to col_1 once reach num_rows
    
    #rgb values for each center pixel
    return [list(pic1.getpixel(i)) for i in center_px_coords]

In [3]:
demo_pic1 = './prac_images/bibzetmoi-small.jpg'
num_rows = 37
num_cols = 49

p1_centerpx_rgbs = img1_tileRGBs(demo_pic1, num_cols, num_rows)

#### Part II
Main modification from prior versions: account for oversaturation -> when the correct _average_ rgb value can't be achieved with simple addition due to 0 & 255 capping.
  
Code for total_avg adapted from [Sp3000's answer here](https://codegolf.stackexchange.com/questions/53621/force-an-average-on-an-image)

In [4]:
def total_avg(img_path=False, local_object=False):
    if local_object:
        width, height = local_object.size
        img = local_object
        
    elif img_path:
        img = pil.Image.open(img_path).convert('RGB')
        width, height = img.size
    
    total_rgb = [0, 0, 0]
    
    for x in range(width):
        for y in range(height):
            for i in range(3): #rgb channels
                total_rgb[i] += img.getpixel((x,y))[i]
                
    return [i / (height*width) for i in total_rgb]

In [5]:
demo_pic2 = './prac_images/source_img2.png'

Ri, Gi, Bi = total_avg(demo_pic2)
print(Ri, Gi, Bi) #leave as floats here

116.25749448123621 103.05903605592347 129.04709713024283


In [13]:
def all_the_exports(img_path, out_path, num_cols, num_rows, reference_rgbs):
    col = 1 #counters for export filenames
    row = 1
    total = 1
    
    for i in reference_rgbs:
        Rf, Gf, Bf = i
        Ri, Gi, Bi = total_avg(img_path = img_path, local_object=False)
        Rmod, Gmod, Bmod = Rf-Ri, Gf-Gi, Bf-Bi
        
        #when you edit values in the pixel_map, it is changed the source image
        #thus must reload source image each iteration to reset the modified pixels
        pic2 = pil.Image.open(img_path).convert('RGB')
        p2_pixelmap = pic2.load()
        width2, height2 = pic2.size
        
        while (abs(Rmod) > 0.5) or (abs(Gmod) > 0.5) or (abs(Bmod) > 0.5):

            for x in range(width2):
                for y in range(height2):
                    p2_pixelmap[x,y] = (int(round(p2_pixelmap[x,y][0]+Rmod,0)), 
                                        int(round(p2_pixelmap[x,y][1]+Gmod,0)), 
                                        int(round(p2_pixelmap[x,y][2]+Bmod,0)))
                #pixelmap defacto restricted to 0-255. No risk of negatives or >255
            Ri2, Gi2, Bi2 = total_avg(img_path=False, local_object = pic2)
            Rmod, Gmod, Bmod = Rf-Ri2, Gf-Gi2, Bf-Bi2
            
        pic2.save(f'{out_path}/{total}_col{col}row{row}.png')
        total += 1
        row += 1
        if (row-1) % num_rows == 0:
            row = 1
            col += 1

In [14]:
out_path = './prac_images/test_export_sat3'

all_the_exports(demo_pic2, out_path, num_cols, num_rows, p1_centerpx_rgbs)

#### QC

In [16]:
def img1_qc(path, num_cols, num_rows): #same as img1_tilesRGB, modified output
    pic1 = pil.Image.open(path).convert('RGB')
    width1, height1 = pic1.size
    def variable_steps(max_pixels, row_or_col):
        px_per_dim = max_pixels/row_or_col #not rounded
        tups = []
        start = 0
        end = start + px_per_dim #not rounded
        for i in range(0,max_pixels+1):
            tups.append( ( int(round(start,0)), int(round(end,0)) ) )
            start = end
            end = start + px_per_dim
            if tups[-1][1] >= max_pixels:
                break
        return tups
    chunk_it_up_cols = variable_steps(width1, num_cols)
    chunk_it_up_rows = variable_steps(height1, num_rows)
    col_centers = [int(round((i[0]+i[1])/2,0)) for i in chunk_it_up_cols]
    row_centers = [int(round((i[0]+i[1])/2,0)) for i in chunk_it_up_rows]
    center_px_coords = [(x,y) for x in col_centers for y in row_centers]    

    return zip(center_px_coords, [list(pic1.getpixel(i)) for i in center_px_coords])

In [17]:
img1_qc_specs = list(img1_qc(demo_pic1, num_cols, num_rows))

In [28]:
print(f'target RGB for center pixel @ {img1_qc_specs[1702][0]} = {img1_qc_specs[1702][1]}')
print('actual avg RGB for corresponding export = ', [round(i, 4) for i in total_avg('./prac_images/test_export_sat1/1703_col47row1.png')])

target RGB for center pixel @ (430, 8) = [178, 198, 233]
actual avg RGB for corresponding export =  [178.0357, 197.7356, 232.789]


In [30]:
print(f'target RGB for center pixel @ {img1_qc_specs[254][0]} = {img1_qc_specs[254][1]}')
print('actual avg RGB for corresponding export = ', [round(i, 4) for i in total_avg('./prac_images/test_export_sat1/255_col7row33.png')])

target RGB for center pixel @ (60, 527) = [0, 7, 3]
actual avg RGB for corresponding export =  [0.4988, 7.271, 3.4852]


In [31]:
print(f'target RGB for center pixel @ {img1_qc_specs[535][0]} = {img1_qc_specs[535][1]}')
print('actual avgRGB for corresponding export = ', [round(i, 4) for i in total_avg('./prac_images/test_export_sat1/536_col15row18.png')])

target RGB for center pixel @ (134, 284) = [23, 18, 0]
actual avgRGB for corresponding export =  [23.4161, 18.2822, 0.4476]
